In [ ]:
import numpy as np
import math
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Kinematic Model definition
class KinematicModel:
    def __init__(self, L, y, theta, Lmax, l, d, p1, sigmaC, p2):
        self.L = L
        self.y = y
        self.theta = theta
        self.Lmax = Lmax
        self.l = l
        self.d = d
        self.p1 = p1
        self.sigmaC = sigmaC
        self.p2 = p2

    def input(self, action):
        noise = norm.rvs(loc=0, scale=action[0] / 4)
        self.y = self.y + action[0] * math.sin(math.radians(action[1] + noise + self.theta))
        nextVehicleEvent = random.uniform(0, 1)
        if nextVehicleEvent < self.p2:
            self.d = self.d + (self.l - action[0] * math.cos(math.radians(action[1] + noise + self.theta)))
        else:
            self.d = 40
        self.theta = self.theta + (action[0] / self.L) * math.tan(math.radians(action[1] + noise))
        curveEvent = random.uniform(0, 1)
        if curveEvent < self.p1:
            curveAngle = norm.rvs(loc=0, scale=self.sigmaC)
            self.theta = self.theta + curveAngle

    def __str__(self):
        return f"y = {self.y} / theta = {self.theta} / d = {self.d}"

# Probability functions (using the provided equations)
def P1(action, environment, sigma, show=True):
    # Compute term1: probability of going out (y > Lmax)
    ratio = (environment.Lmax - environment.y) / action[0]
    if ratio > 1 or ratio < -1:
        term1 = 0
    else:
        angle = math.degrees(math.asin(ratio))
        term1 = 1 - norm.cdf((1 / sigma) * (angle - action[1] - environment.theta))
    # Compute term2: probability of going out (y < -Lmax)
    ratio2 = (-environment.Lmax - environment.y) / action[0]
    if ratio2 > 1 or ratio2 < -1:
        term2 = 0
    else:
        angle = math.degrees(math.asin(ratio2))
        term2 = norm.cdf((1 / sigma) * (angle - action[1] - environment.theta))
    value = (1 - environment.p1) * (term1 + term2)
    if show:
        print(f"P1 Value: {value}")
    return value, term1, term2

def P2(action, environment, sigma, show=True):
    # Similar to P1 but with weight environment.p1
    ratio = (environment.Lmax - environment.y) / action[0]
    if ratio > 1 or ratio < -1:
        term1 = 0
    else:
        angle = math.degrees(math.asin(ratio))
        term1 = 1 - norm.cdf((1 / sigma) * (angle - action[1] - environment.theta))
    ratio2 = (-environment.Lmax - environment.y) / action[0]
    if ratio2 > 1 or ratio2 < -1:
        term2 = 0
    else:
        angle = math.degrees(math.asin(ratio2))
        term2 = norm.cdf((1 / sigma) * (angle - action[1] - environment.theta))
    value = environment.p1 * (term1 + term2)
    if show:
        print(f"P2 Value: {value}")
    return value, term1, term2

def P3(action, environment, sigma, show=True):
    # Probability of risking a crash in a straight road
    if (1 / action[0]) * (environment.l + (((action[0] * 3.6) / -2) + environment.d)) > 1:
        return 0
    angle = math.degrees(math.acos((1 / action[0]) * (environment.l + (((action[0] * 3.6) / -2) + environment.d))))
    value = norm.cdf((1 / sigma) * (angle - abs(action[1]) - abs(environment.theta)))
    value = environment.p2 * (1 - environment.p1) * value
    if show:
        print(f"P3 Value: {value}")
    return value

def P4(action, environment, sigma, show=True):
    # Probability of risking a crash in a curved road
    if (1 / action[0]) * (environment.l + (((action[0] * 3.6) / -2) + environment.d)) > 1:
        return 0
    angle = math.degrees(math.acos((1 / action[0]) * (environment.l + (((action[0] * 3.6) / -2) + environment.d))))
    value = norm.cdf((1 / sigma) * (angle - abs(action[1]) - abs(environment.theta)))
    value = environment.p2 * environment.p1 * value
    if show:
        print(f"P4 Value: {value}")
    return value

# Transition probabilities for different outcomes
def anyToG(action, environment):
    P1_val, _, _ = P1(action, environment, action[0] / 4, show=False)
    P2_val, _, _ = P2(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    term1 = 1 - (P1_val + P2_val)   # probability of staying in-road
    term2 = 1 - (P3(action, environment, action[0] / 4, show=False) + 
                 P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False))
    return term1 * term2

def anyToX(action, environment):
    P1_val, _, _ = P1(action, environment, action[0] / 4, show=False)
    P2_val, _, _ = P2(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    term1 = P1_val + P2_val      # probability of going off-road
    term2 = 1 - (P3(action, environment, action[0] / 4, show=False) + 
                 P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False))
    return term1 * term2

def anyToI(action, environment):
    P1_val, _, _ = P1(action, environment, action[0] / 4, show=False)
    P2_val, _, _ = P2(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    term1 = 1 - (P1_val + P2_val)
    term2 = (P3(action, environment, action[0] / 4, show=False) + 
             P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False))
    return term1 * term2

def anyToXI(action, environment):
    P1_val, _, _ = P1(action, environment, action[0] / 4, show=False)
    P2_val, _, _ = P2(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    term1 = P1_val + P2_val
    term2 = (P3(action, environment, action[0] / 4, show=False) + 
             P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False))
    return term1 * term2

# Build transition probability matrix using the passed environment
def transitionProbabilitiesMatrix(actions, environment):
    transitions = [anyToG, anyToX, anyToI, anyToXI]
    transitionMatrix = np.array([
        [transition(action, environment) for transition in transitions] 
         for action in actions
    ])
    return transitionMatrix

# Reward functions
def rewardCenterProbability(action, environment, ratio, show=True):
    # Compute probability of staying near the center
    # Use parentheses to avoid ambiguity: 1/(action[0]/4) is the same as 4/action[0]
    numerator1 = (environment.Lmax * ratio) - environment.y
    if abs(numerator1 / action[0]) > 1:
        term1 = term1C = 0
    else:
        angle = math.degrees(math.asin(numerator1 / action[0]))
        term1 = norm.cdf((4 / action[0]) * (angle - action[1] - environment.theta))
        term1C = norm.cdf((1 / math.sqrt(action[0] / 4 + environment.sigmaC)) * (angle - action[1] - environment.theta))
        
    numerator2 = (-environment.Lmax * ratio) - environment.y
    if abs(numerator2 / action[0]) > 1:
        term2 = term2C = 0
    else:
        angle = math.degrees(math.asin(numerator2 / action[0]))
        term2 = norm.cdf((4 / action[0]) * (angle - action[1] - environment.theta))
        term2C = norm.cdf((1 / math.sqrt(action[0] / 4 + environment.sigmaC)) * (angle - action[1] - environment.theta))
        
    value = ((1 - environment.p1) * (term1 - term2)) + (environment.p1 * (term1C - term2C))
    if show:
        print(f"Reward Center Probability Value: {value}")
    return value

def rewardSpeed(action, environment):
    return -0.5 * abs(environment.l - action[0])

def rewardDistanceProbability(action, environment):
    term1 = P3(action, environment, action[0] / 4, show=False)
    term2 = P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    return term1 + term2

def expectedReward(action, environment, r1, r2, r3, r4, show=True):
    termR1 = r1 * rewardCenterProbability(action, environment, 0.5, show=False)
    termR2 = r2 * rewardCenterProbability(action, environment, 0.25, show=False)
    rSpeed = rewardSpeed(action, environment)
    termR3 = r3 * (1 - rewardDistanceProbability(action, environment))
    termR4 = r4 * rewardDistanceProbability(action, environment)
    total_reward = termR1 + termR2 + rSpeed + termR3 + termR4
    if show:
        print(f"Reward for action {action}: {total_reward}")
    return total_reward

def expectedRewardMatrix(actions, environment, r1, r2, r3, r4):
    rewardMatrix = np.array([
        expectedReward(action, environment, r1, r2, r3, r4, show=False) 
        for action in actions
    ])
    return rewardMatrix

# Define the CMDP environment parameters:
kinematicModel = KinematicModel(L=1, y=0, theta=0, Lmax=2, l=25, d=40, p1=0.05, sigmaC=5, p2=0.3)

# Define states and actions
states = ['G', 'X', 'I', 'XI']  # S_G: good, S_X: off-road, S_I: risk of crash, S_XI: off-road and crash risk
anglesDeg = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
velocities = [20, 25, 30]
actions = [(v, delta) for v in velocities for delta in anglesDeg]

# Build the (fixed) transition and reward matrices for the CMDP
T = transitionProbabilitiesMatrix(actions, kinematicModel)
r1, r2, r3, r4 = 50, 100, -1, 0
R = expectedRewardMatrix(actions, kinematicModel, r1, r2, r3, r4)

# Transition function for the MDP
def transition_func(s, a, T):
    action_index = actions.index(a)
    probabilities = T[action_index]
    return [(state, prob) for state, prob in zip(states, probabilities)]

# Reward function for the MDP
def reward_func(s, a, s_prime, R):
    action_index = actions.index(a)
    return R[action_index]

# --- Value Iteration ---
def value_iteration(states, actions, transition_func, reward_func, environment, gamma=0.9, theta=1e-4):
    """
    Perform value iteration on a fixed CMDP.
    """
    # Initialize V(s) = 0 for all states and empty policy
    V = {s: 0 for s in states}
    policy = {s: None for s in states}

    # Use fixed transition and reward matrices computed from the environment
    T = transitionProbabilitiesMatrix(actions, environment)
    r1, r2, r3, r4 = 50, 100, -1, 0
    R = expectedRewardMatrix(actions, environment, r1, r2, r3, r4)

    iteration = 0
    while True:
        delta = 0
        print(f"--- Iteration {iteration} ---")
        iteration += 1
        for s in states:
            v_old = V[s]
            action_values = {}
            for a in actions:
                expected_value = 0
                for s_prime, p in transition_func(s, a, T):
                    expected_value += p * (reward_func(s, a, s_prime, R) + gamma * V[s_prime])
                action_values[a] = expected_value
            best_action_value = max(action_values.values())
            best_action = max(action_values, key=action_values.get)
            V[s] = best_action_value
            policy[s] = best_action
            print(f"State {s}: Best action {best_action} with value {best_action_value:.2f}")
            delta = max(delta, abs(v_old - V[s]))
        if delta < theta:
            print("Convergence reached.")
            break
    return V, policy

# Run value iteration on the fixed model
V, policy = value_iteration(states, actions, transition_func, reward_func, kinematicModel, gamma=0.9, theta=1e-4)

print("\nFinal Value Function:")
for state, value in V.items():
    print(f"{state}: {value:.2f}")

print("\nFinal Policy:")
for state, action in policy.items():
    print(f"{state}: {action}")


--- Iteration 0 ---
State G: Best action (20, 0) with value 41.85
State X: Best action (20, 0) with value 70.40
State I: Best action (20, 0) with value 85.73
State XI: Best action (20, 0) with value 85.73
--- Iteration 1 ---
State G: Best action (25, 0) with value 86.89
State X: Best action (20, 0) with value 116.46
State I: Best action (20, 0) with value 126.49
State XI: Best action (20, 0) with value 126.49
--- Iteration 2 ---
State G: Best action (25, 0) with value 126.56
State X: Best action (20, 0) with value 153.56
State I: Best action (20, 0) with value 161.64
State XI: Best action (20, 0) with value 161.64
--- Iteration 3 ---
State G: Best action (20, 0) with value 161.64
State X: Best action (20, 0) with value 185.56
State I: Best action (20, 0) with value 192.53
State XI: Best action (20, 0) with value 192.53
--- Iteration 4 ---
State G: Best action (20, 0) with value 192.53
State X: Best action (20, 0) with value 213.61
State I: Best action (20, 0) with value 219.72
State XI

Considering more MDP with different intial states 

In [2]:
import numpy as np
import math
import random
from scipy.stats import norm
import matplotlib.pyplot as plt

# --- Kinematic Model and CMDP Functions (same as in the corrected code) ---

class KinematicModel:
    def __init__(self, L, y, theta, Lmax, l, d, p1, sigmaC, p2):
        self.L = L
        self.y = y
        self.theta = theta
        self.Lmax = Lmax
        self.l = l
        self.d = d
        self.p1 = p1
        self.sigmaC = sigmaC
        self.p2 = p2

    def input(self, action):
        noise = norm.rvs(loc=0, scale=action[0] / 4)
        self.y = self.y + action[0] * math.sin(math.radians(action[1] + noise + self.theta))
        nextVehicleEvent = random.uniform(0, 1)
        if nextVehicleEvent < self.p2:
            self.d = self.d + (self.l - action[0] * math.cos(math.radians(action[1] + noise + self.theta)))
        else:
            self.d = 40
        self.theta = self.theta + (action[0] / self.L) * math.tan(math.radians(action[1] + noise))
        curveEvent = random.uniform(0, 1)
        if curveEvent < self.p1:
            curveAngle = norm.rvs(loc=0, scale=self.sigmaC)
            self.theta = self.theta + curveAngle

    def __str__(self):
        return f"y = {self.y} / theta = {self.theta} / d = {self.d}"

def P1(action, environment, sigma, show=True):
    ratio = (environment.Lmax - environment.y) / action[0]
    if ratio > 1 or ratio < -1:
        term1 = 0
    else:
        angle = math.degrees(math.asin(ratio))
        term1 = 1 - norm.cdf((1 / sigma) * (angle - action[1] - environment.theta))
    ratio2 = (-environment.Lmax - environment.y) / action[0]
    if ratio2 > 1 or ratio2 < -1:
        term2 = 0
    else:
        angle = math.degrees(math.asin(ratio2))
        term2 = norm.cdf((1 / sigma) * (angle - action[1] - environment.theta))
    value = (1 - environment.p1) * (term1 + term2)
    if show:
        print(f"P1 Value: {value}")
    return value, term1, term2

def P2(action, environment, sigma, show=True):
    ratio = (environment.Lmax - environment.y) / action[0]
    if ratio > 1 or ratio < -1:
        term1 = 0
    else:
        angle = math.degrees(math.asin(ratio))
        term1 = 1 - norm.cdf((1 / sigma) * (angle - action[1] - environment.theta))
    ratio2 = (-environment.Lmax - environment.y) / action[0]
    if ratio2 > 1 or ratio2 < -1:
        term2 = 0
    else:
        angle = math.degrees(math.asin(ratio2))
        term2 = norm.cdf((1 / sigma) * (angle - action[1] - environment.theta))
    value = environment.p1 * (term1 + term2)
    if show:
        print(f"P2 Value: {value}")
    return value, term1, term2

def P3(action, environment, sigma, show=True):
    if (1 / action[0]) * (environment.l + (((action[0] * 3.6) / -2) + environment.d)) > 1:
        return 0
    angle = math.degrees(math.acos((1 / action[0]) * (environment.l + (((action[0] * 3.6) / -2) + environment.d))))
    value = norm.cdf((1 / sigma) * (angle - abs(action[1]) - abs(environment.theta)))
    value = environment.p2 * (1 - environment.p1) * value
    if show:
        print(f"P3 Value: {value}")
    return value

def P4(action, environment, sigma, show=True):
    if (1 / action[0]) * (environment.l + (((action[0] * 3.6) / -2) + environment.d)) > 1:
        return 0
    angle = math.degrees(math.acos((1 / action[0]) * (environment.l + (((action[0] * 3.6) / -2) + environment.d))))
    value = norm.cdf((1 / sigma) * (angle - abs(action[1]) - abs(environment.theta)))
    value = environment.p2 * environment.p1 * value
    if show:
        print(f"P4 Value: {value}")
    return value

def anyToG(action, environment):
    P1_val, _, _ = P1(action, environment, action[0] / 4, show=False)
    P2_val, _, _ = P2(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    term1 = 1 - (P1_val + P2_val)
    term2 = 1 - (P3(action, environment, action[0] / 4, show=False) +
                 P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False))
    return term1 * term2

def anyToX(action, environment):
    P1_val, _, _ = P1(action, environment, action[0] / 4, show=False)
    P2_val, _, _ = P2(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    term1 = P1_val + P2_val
    term2 = 1 - (P3(action, environment, action[0] / 4, show=False) +
                 P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False))
    return term1 * term2

def anyToI(action, environment):
    P1_val, _, _ = P1(action, environment, action[0] / 4, show=False)
    P2_val, _, _ = P2(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    term1 = 1 - (P1_val + P2_val)
    term2 = (P3(action, environment, action[0] / 4, show=False) +
             P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False))
    return term1 * term2

def anyToXI(action, environment):
    P1_val, _, _ = P1(action, environment, action[0] / 4, show=False)
    P2_val, _, _ = P2(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    term1 = P1_val + P2_val
    term2 = (P3(action, environment, action[0] / 4, show=False) +
             P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False))
    return term1 * term2

def transitionProbabilitiesMatrix(actions, environment):
    transitions = [anyToG, anyToX, anyToI, anyToXI]
    transitionMatrix = np.array([
        [transition(action, environment) for transition in transitions]
        for action in actions
    ])
    return transitionMatrix

def rewardCenterProbability(action, environment, ratio, show=True):
    numerator1 = (environment.Lmax * ratio) - environment.y
    if abs(numerator1 / action[0]) > 1:
        term1 = term1C = 0
    else:
        angle = math.degrees(math.asin(numerator1 / action[0]))
        term1 = norm.cdf((4 / action[0]) * (angle - action[1] - environment.theta))
        term1C = norm.cdf((1 / math.sqrt(action[0] / 4 + environment.sigmaC)) * (angle - action[1] - environment.theta))
    numerator2 = (-environment.Lmax * ratio) - environment.y
    if abs(numerator2 / action[0]) > 1:
        term2 = term2C = 0
    else:
        angle = math.degrees(math.asin(numerator2 / action[0]))
        term2 = norm.cdf((4 / action[0]) * (angle - action[1] - environment.theta))
        term2C = norm.cdf((1 / math.sqrt(action[0] / 4 + environment.sigmaC)) * (angle - action[1] - environment.theta))
    value = ((1 - environment.p1) * (term1 - term2)) + (environment.p1 * (term1C - term2C))
    if show:
        print(f"Reward Center Probability Value: {value}")
    return value

def rewardSpeed(action, environment):
    return -0.5 * abs(environment.l - action[0])

def rewardDistanceProbability(action, environment):
    term1 = P3(action, environment, action[0] / 4, show=False)
    term2 = P4(action, environment, math.sqrt(action[0] / 4 + environment.sigmaC), show=False)
    return term1 + term2

def expectedReward(action, environment, r1, r2, r3, r4, show=True):
    termR1 = r1 * rewardCenterProbability(action, environment, 0.5, show=False)
    termR2 = r2 * rewardCenterProbability(action, environment, 0.25, show=False)
    rSpeed = rewardSpeed(action, environment)
    termR3 = r3 * (1 - rewardDistanceProbability(action, environment))
    termR4 = r4 * rewardDistanceProbability(action, environment)
    total_reward = termR1 + termR2 + rSpeed + termR3 + termR4
    if show:
        print(f"Reward for action {action}: {total_reward}")
    return total_reward

def expectedRewardMatrix(actions, environment, r1, r2, r3, r4):
    rewardMatrix = np.array([
        expectedReward(action, environment, r1, r2, r3, r4, show=False)
        for action in actions
    ])
    return rewardMatrix

# Define states and actions (as before)
states = ['G', 'X', 'I', 'XI']
anglesDeg = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
velocities = [20, 25, 30]
actions = [(v, delta) for v in velocities for delta in anglesDeg]

# Transition and reward functions for the MDP:
def transition_func(s, a, T):
    action_index = actions.index(a)
    probabilities = T[action_index]
    return [(state, prob) for state, prob in zip(states, probabilities)]

def reward_func(s, a, s_prime, R):
    action_index = actions.index(a)
    return R[action_index]

def value_iteration(states, actions, transition_func, reward_func, environment, gamma=0.9, theta=1e-4):
    V = {s: 0 for s in states}
    policy = {s: None for s in states}
    T = transitionProbabilitiesMatrix(actions, environment)
    r1, r2, r3, r4 = 50, 100, -1, 0
    R = expectedRewardMatrix(actions, environment, r1, r2, r3, r4)
    
    iteration = 0
    while True:
        delta = 0
        print(f"--- Iteration {iteration} ---")
        iteration += 1
        for s in states:
            v_old = V[s]
            action_values = {}
            for a in actions:
                expected_value = 0
                for s_prime, p in transition_func(s, a, T):
                    expected_value += p * (reward_func(s, a, s_prime, R) + gamma * V[s_prime])
                action_values[a] = expected_value
            best_action_value = max(action_values.values())
            best_action = max(action_values, key=action_values.get)
            V[s] = best_action_value
            policy[s] = best_action
            print(f"State {s}: Best action {best_action} with value {best_action_value:.2f}")
            delta = max(delta, abs(v_old - V[s]))
        if delta < theta:
            print("Convergence reached.")
            break
    return V, policy

# --- End of CMDP Definitions ---

# --- Create Many Small Stationary MDPs ---

# For example, we vary the initial y (lateral position) and theta (heading angle)
initial_conditions = [
    (y, theta) for y in np.linspace(-1, 1, 5) for theta in np.linspace(-5, 5, 5)
]

results = {}

for (y, theta) in initial_conditions:
    # Instantiate a new kinematic model with the given initial state.
    km = KinematicModel(L=1, y=y, theta=theta, Lmax=2, l=25, d=40, p1=0.05, sigmaC=5, p2=0.3)
    print(f"\n=== Running Value Iteration for initial state: y = {y:.2f}, theta = {theta:.2f} ===")
    V, policy = value_iteration(states, actions, transition_func, reward_func, km, gamma=0.9, theta=1e-4)
    results[(y, theta)] = (V, policy)

# --- Analyze or Visualize the Results ---
# For example, print the resulting value function and policy for each starting condition.
for (y, theta), (V, policy) in results.items():
    print(f"\nInitial state y = {y:.2f}, theta = {theta:.2f}")
    print("Value Function:", V)
    print("Policy:", policy)



=== Running Value Iteration for initial state: y = -1.00, theta = -5.00 ===
--- Iteration 0 ---
State G: Best action (20, 5) with value 34.92
State X: Best action (20, 5) with value 56.34
State I: Best action (20, 5) with value 72.49
State XI: Best action (20, 5) with value 72.49
--- Iteration 1 ---
State G: Best action (25, 5) with value 75.16
State X: Best action (20, 5) with value 97.18
State I: Best action (20, 5) with value 108.88
State XI: Best action (20, 5) with value 108.88
--- Iteration 2 ---
State G: Best action (25, 5) with value 110.52
State X: Best action (20, 5) with value 130.57
State I: Best action (20, 5) with value 140.14
State XI: Best action (20, 5) with value 140.14
--- Iteration 3 ---
State G: Best action (25, 5) with value 140.64
State X: Best action (20, 5) with value 158.61
State I: Best action (20, 5) with value 166.65
State XI: Best action (20, 5) with value 166.65
--- Iteration 4 ---
State G: Best action (20, 5) with value 166.65
State X: Best action (20, 